# Visualização Cartográfica

_“A criação de mapas é um dos atividades intelectuais mais antigos da humanidade e também uma das mais complexas, combinando teoria científica, representação gráfica, fatos geográficos e considerações práticas de diversas maneiras.”_ &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

A cartografia &ndash; o estudo e a prática da criação de mapas &ndash; tem uma história rica em descobertas e designs ao longo dos séculos. A visualização cartográfica utiliza técnicas de mapeamento para representar dados contendo informações espaciais, como localizações, rotas ou trajetórias na superfície da Terra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>


Ao aproximarmos a Terra para uma esfera, podemos denotar posições utilizando um sistema de coordenadas esféricas baseado em _latitude_ (ângulo em graus indo ao norte ou ao sul do _equador_) e _longitude_ (ângulo em graus que especifica a posição leste-oeste). Nesse sistema, um _paralelo_ é um círculo de latitude constante, enquanto um _meridiano_ é um círculo de longitude constante. O [_Meridiano de Greenwich_](https://en.wikipedia.org/wiki/Prime_meridian) está localizado na longitude 0° e, por convenção, passa pelo Observatório Real de Greenwich, na Inglaterra.

Para "achatar" uma esfera tridimensional em um plano bidimensional, é necessário aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeie pares de coordenadas (`longitude`, `latitude`) para (`x`, `y`). De forma semelhante às [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções transformam um domínio de dados (posição espacial) em um intervalo visual (posição em pixels). No entanto, as escalas que vimos até agora aceitam um domínio unidimensional, enquanto as projeções cartográficas são inerentemente bidimensionais.

Neste notebook, introduziremos os conceitos básicos da criação de mapas e da visualização de dados espaciais com Altair, incluindo:

- Formatos de dados para representar elementos geográficos,
- Técnicas de geo-visualização, como mapas de pontos, símbolos e mapas coropléticos, e
- Uma revisão das projeções cartográficas mais comuns.

_Este notebook faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._


In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

## Dados Geográficos: GeoJSON e TopoJSON

Até agora, trabalhamos com conjuntos de dados nos formatos JSON e CSV, que correspondem a tabelas de dados compostas por linhas (registros) e colunas (campos). No entanto, para representar regiões geográficas (países, estados, _etc._) e trajetórias (rotas de voo, linhas de metrô, _etc._), precisamos expandir nosso repertório com formatos adicionais projetados para suportar geometrias complexas.

O [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela elementos geográficos dentro de um formato especializado de JSON. Um `feature` do GeoJSON pode incluir dados geométricos &ndash; como coordenadas de `longitude` e `latitude` que definem o contorno de um país &ndash; além de atributos adicionais dos dados.

Aqui está um objeto `feature` do GeoJSON que representa o contorno do estado do Colorado, nos Estados Unidos:


~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

O `feature` inclui um objeto `properties`, que pode inlcuir qualquer número de campos de dados, além de um objeto `geometry`, que, neste caso, contém um único polígono formado por coordenadas `[longitude, latitude]` que definem o contorno do estado. As coordenadas continuam à direita por um bom trecho, caso queira continuar rolando o scroll do mouse...

Para saber mais sobre os detalhes técnicos do GeoJSON, consulte a [especificação oficial do GeoJSON](http://geojson.org/) ou leia a [introdução útil de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).


Uma desvantagem do GeoJSON como formato de armazenamento é que ele pode ser redundante, resultando em arquivos de tamanhos maiores. Considere: o Colorado compartilha fronteiras com seis outros estados (sete, se incluirmos o ponto de contato com o Arizona). Ao invés de usar listas de coordenadas sobrepostas para cada um desses estados, uma abordagem mais esperta é codificar as fronteiras compartilhadas apenas uma vez, representando a _topologia_ das regiões geográficas. Felizmente, é exatamente isso que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!


Vamos carregar um arquivo TopoJSON de países do mundo (numa resolução de 110 metros):

In [ ]:
world = data.world_110m.url
world

In [ ]:
world_topo = data.world_110m()

In [ ]:
world_topo.keys()

In [ ]:
world_topo['type']

In [ ]:
world_topo['objects'].keys()

_Inspecione o dicionário TopoJSON `world_topo` acima para ver seu conteúdo._

Nos dados acima, a propriedade `objects` indica os elementos nomeados que podemos extrair: geometrias para todos os `countries` (países) ou um único polígono representando toda a superfície terrestre (`land`). Qualquer um desses elementos pode ser convertido para dados GeoJSON que podemos futuramente visualizar.

Como o TopoJSON é um formato especializado, precisamos instruir o Altair a interpretar corretamente o formato TopoJSON, indicando qual objeto nomeado queremos extrair da topologia. O código a seguir indica que queremos extrair os elementos GeoJSON do dataset `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

Essa chamada ao método `alt.topo_feature` se expande para o seguinte JSON do Vega-Lite:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que podemos carregar dados geográficos, estamos prontos para começar a fazer mapas!
